<a href="https://colab.research.google.com/github/leenago/NLP_edu/blob/master/200903_TDM_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TDM 구현하기

In [1]:
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']

In [2]:
doc_list = []
for doc in docs:
  doc_list.append(doc.split(' '))

doc_list

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [6]:
from collections import defaultdict
word2id = defaultdict(lambda: len(word2id))
for doc in doc_list:
  for token in doc:
    word2id[token]

word2id

defaultdict(<function __main__.<lambda>>,
            {'동물원': 0, '바나나': 3, '아기': 5, '엄마': 4, '원숭이': 2, '코끼리': 1})

In [8]:
import numpy as np

TDM = np.zeros((len(word2id), len(doc_list)), dtype=int)
print(TDM)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [9]:
#1차원인 BoW 때와 달리, TDM은 list에 append하지 않는 것이 차이이다!
for i, doc in enumerate(doc_list):
  for token in doc:
    TDM[word2id[token], i] += 1 #해당 토큰의 위치(column)

#행렬로 표기(BoW와 차이점: BoW는 1차원 배열이다.)
TDM

array([[1, 1, 0, 0],
       [1, 0, 2, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [11]:
import pandas as pd

doc_names = ['문서'+str(i) for i in range(len(doc_list))]
#여러줄 표기를 한 줄로 옮긴 것, 해석은 뒤에서부터 하면 된다.
print('doc_names: ', doc_names)
print('\n')
sorted_vocab = sorted((value,key) for key, value in word2id.items())
vocab = [v[1] for v in sorted_vocab]

df_TDM = pd.DataFrame(TDM, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')


doc_names:  ['문서0', '문서1', '문서2', '문서3']




,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


#TDM - sklearn
- sklearn은 DTM으로 만들어지게 설정해준다.
('.T'가 필요하다)
- CountVectorizer는 docs를 알아서 token화 해주고 count를 해준다

In [12]:
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

DTM = count_vect.fit_transform(docs)
DTM.toarray()

array([[1, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 2],
       [0, 2, 0, 0, 1, 1]])

In [14]:
DTM.toarray().T

array([[1, 1, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 2, 1]])

In [15]:
import pandas as pd

doc_names = ['문서'+str(i) for i in range(len(doc_list))]
vocab = count_vect.get_feature_names()
print(vocab)
print('\n')

df_TDM = pd.DataFrame(DTM.toarray().T, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

['동물원', '바나나', '아기', '엄마', '원숭이', '코끼리']




,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
바나나,0,1,0,2
아기,0,0,1,0
엄마,0,0,1,0
원숭이,0,1,0,1
코끼리,1,0,2,1


#TDM-gensim

In [40]:
docs = ['동물원 코끼리',
        '동물원 원숭이 바나나',
        '엄마 코끼리 아기 코끼리',
        '원숭이 바나나 코끼리 바나나']

In [41]:
doc_list = []
for doc in docs:
  doc_list.append(doc.split(' '))

doc_list

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [42]:
from collections import defaultdict
word2id = defaultdict(lambda: len(word2id))

for doc in doc_list:
  for token in doc:
    word2id[token]

word2id

defaultdict(<function __main__.<lambda>>,
            {'동물원': 0, '바나나': 3, '아기': 5, '엄마': 4, '원숭이': 2, '코끼리': 1})

In [43]:
import gensim
from gensim import corpora

doc_list = [doc.split() for doc in docs] #for문 한줄로 구현하기, 뒤에서부터 해석한다.

id2word = corpora.Dictionary(doc_list)
TDM = [id2word.doc2bow(doc) for doc in doc_list]
TDM


[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

In [49]:
from gensim.matutils import sparse2full
import pandas as pd
import numpy as np

doc_names = ['문서' + str(i) for i in range(len(doc_list))]
vocab = [id2word[i] for i in id2word.keys()]
DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in TDM]

df_TDM = pd.DataFrame(np.array(DTM_matrix, dtype = int).T,
                      columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0
